# 使用 `Scene` 读取数据
Satpy 处理数据的主要接口是 `Scene` 类。我们可以向 `Scene` 提供数据文件，并使用“读取器”加载它们。在本笔记本中，我们将探讨 Satpy 提供的基本数据加载和数据访问功能，同时简要介绍 xarray 的 `DataArray` 对象和 `dask` 数组。
在导入和使用Satpy之前，我们运行一些Python代码来进行初始设置。这包括关闭警告和限制我们使用的资源数量。这些预防措施是为了确保这些示例在大多数机器上都能正常运行。

In [ ]:
%run ../init_notebook.py
from satpy import Scene
from glob import glob

# Get the list of GOES-16 ABI files to open
filenames = glob('../data/abi_l1b/20180511_texas_fire_abi_l1b_conus/*.nc')
len(filenames)

In [ ]:
scn = Scene(reader='abi_l1b', filenames=filenames)
scn.keys()

我们现在已创建了一个 `Scene` 对象。在幕后，Satpy 已整理了文件并确定了我们可以访问的内容。我们实际上还没有加载任何数据，因此我们的类似字典的 `Scene` 对象是空的。要找出可以从文件中加载哪些数据，我们可以使用 `available_dataset_names`。

In [ ]:
scn.available_dataset_names()

`Scene` 告诉我们我们有所有 16 个 ABI 通道可供加载。此列表包括任何可以通过 "abi_l1b" 读取器配置访问的文件加载的产品。如果我们未提供所有必要的文件，或者由于某些原因文件中缺失了数据，那么该产品将不会出现在此处列表中。
| 通道     | 波长  | 分辨率  || ----------- | ----------- |  ----------- |
| C01         | 0.47微米      |  1000米       || C02         | 0.64微米      |  500米        || C03         | 0.86微米      |  1000米       || C04         | 1.37微米      |  2000米       || C05         | 1.60µm      |  1000m       || C06         | 2.20微米      |  2000米       || 项目     | 波长      | 范围     |
|----------|-----------|----------|
| C07      | 3.90微米   | 2000米   || C08         | 6.20µm      |  2000m       || C09         | 6.90µm      |  2000m       || C10         | 7.30微米      |  2000米       || C11         | 8.40µm      |  2000m       || C12 | 9.60µm | 2000m || C13         | 10.30微米     |  2000米       || C14         | 11.20微米     |  2000米       || C15         | 12.30微米     |  2000米       || C16         | 13.30微米     |  2000米       |
让我们选择其中一个通道，加载它，并查看Satpy所提供的信息。

In [ ]:
my_channel = 'EDITME'
scn.load([my_channel])
# use brackets to access products like a normal dict
scn[my_channel]

## Xarray 和 Dask
上方显示了一个带有大量元数据的 `xarray.DataArray` 对象。在使用 Satpy 的 DataArray 时，有几个需要熟悉的元素：
* `dask.array<...>`: 我们看不到任何实际的影像数据。我们的数据存储在 `dask` 数组中，而不是传统的 NumPy 数组。这意味着我们的数据加载和计算是延迟进行的。* `属性`: 一个用于存储元数据的字典。其中一部分来自文件，另一部分由 "abi_l1b" 读取器添加，以协助后续的 Satpy 操作。其中一些更为重要的键包括：
* `平台名称`* `传感器`* `名称`* `波长`* `单位`* `校准`* `standard_name`* `开始时间`* `区域` (稍后将详细介绍)
如果我们想访问属性，我们会使用`.attrs`属性。

In [ ]:
scn[my_channel].attrs['start_time']

我们可以使用 `.dims` 属性来访问数据的维度名称。

In [ ]:
scn[my_channel].dims

那些维度的尺寸：

In [ ]:
scn[my_channel].sizes['y']

DataArrays 也提供对传统 NumPy 属性如 `shape` 和 `ndim` 的访问。

In [ ]:
scn[my_channel].shape

In [ ]:
scn[my_channel].ndim

尽管通常不需要，但可以通过xarray的`DataArray`的`.data`属性访问其底层的dask数组。

In [ ]:
scn[my_channel].data

## 延迟计算
通常不需要直接访问 dask 数组，因为 xarray 会为我们处理所有常规的算术运算和 NumPy 函数。我们可以像处理普通的 Python 变量一样处理这些数组，进行加减运算，并将结果存储在一个新变量中。
以一个任意的例子，让我们将 `2.5` 加到我们已加载的通道中，并将其存储到新变量 `my_new_var` 中：

In [ ]:
my_new_var = scn[my_channel] + 2.5

存储在 `Scene` 中的 `DataArray` 对象不受影响，但我们的变量 `my_new_var` 确实包含了此次计算的更改。默认情况下，Xarray 会尽可能保留维度和坐标，但会丢失所有属性。

In [ ]:
my_new_var

这里的一个重要点是，由于涉及延迟的Dask操作，这些操作看起来似乎很快。实际上我们还没有进行计算。即使我们已经修改了数据，我们仍然有一个表示这些变化的Dask数组。如果我们习惯于普通的NumPy数组，通常预期会立即得到结果，这有时会使我们的数据分析变得有些令人困惑。

In [ ]:
scn[my_channel].max()

在这些情况下，我们可以使用 `.compute()` 方法来加载数据并执行到目前为止构建的一系列计算。Dask 会将数据拆分到多个线程中以并行计算值；为我们处理所有多线程和低级同步问题。让我们计算存储在 `Scene` 中的数据以及我们新数据变量的最大值：

In [ ]:
scn[my_channel].max().compute()

In [ ]:
my_new_var.max().compute()

请注意，此操作仍会返回一个 `DataArray`，因此我们可以访问任何剩余的坐标或维度。如果希望返回普通的 NumPy 数组，可以使用 `.values` 来计算 dask 数组并返回结果 NumPy 数组。

In [ ]:
scn[my_channel].max().values
# we could also do:
# scn[my_channel].max().compute().data

## 像numpy数组一样使用它们
在大多数情况下，Xarray 的 `DataArray` 对象可以像常规的 numpy 数组一样使用。当需要实际数据值时，它们会被计算出来。这使我们能够几乎无需额外工作即可将 `DataArray` 对象与其他 Python 工具一起使用。然后进行一个简单的 matplotlib 绘图以查看我们的数据。
<sub>注意：如果在 Jupyter Lab 会话中运行，可能需要将下面单元格中的 "notebook" 更改为 "inline"。</sub>

In [ ]:
%matplotlib notebook

import matplotlib.pyplot as plt
plt.figure()
plt.imshow(scn[my_channel])
plt.colorbar()

我们可以像上面那样手动使用 matplotlib 的调用，但 Xarray 也提供了自己的绘图实用函数来简化这一过程。

In [ ]:
plt.figure()
scn[my_channel].plot.imshow(cmap='viridis')

如果我们比较DataArray的`.attrs`中的元数据与图表上的标签，就可以看到Xarray在命名图表组件时所做的最佳猜测。它使用了如`long_name`这样的属性来为颜色条命名，并使用维度名称作为坐标轴标签。Xarray的绘图工具本质上是对matplotlib的简单封装，因此我们仍然可以访问matplotlib的所有功能。例如，我们可以在上述代码单元中添加常见的matplotlib函数调用，如`plt.title(my_channel)`，以更改标题。
我们也可以通过向 `imshow` 调用传递 `cmap` 关键字参数来更改颜色映射（例如 `cmap='viridis'`）。要查看完整的内置 matplotlib 颜色映射列表，请参见 [matplotlib文档](https://matplotlib.org/tutorials/colors/colormaps.html)。默认情况下，matplotlib 会使用 `viridis`，但我们也可以尝试其他颜色映射，如 `plasma`、`magma`、`RdBu_r`、`Reds` 或 `tab20b`。

## 切片
就像 numpy 数组一样，我们也可以对 `DataArray` 进行切片以获取特定区域的数据。我们将使用索引切片和步长来展示一个更小的区域以及通道 2 (`'C02'`) 产品的较低分辨率。请注意切片操作不会移除 DataArray 的属性。切片语法为 `start_index:end_index:stride`，其中起始索引是包含的，结束索引是不包含的，而步长表示每隔 X 个像素取一个。

In [ ]:
# load C02 if we haven't already
scn.load(['C02'])

# slice the DataArray and print out its representation
scn['C02'][2500:3500:4, 1500:2500:4]

In [ ]:
plt.figure()  # create a new figure
scn['C02'][2500:3500:4, 1500:2500:4].plot.imshow()

### 练习
**时间：5-10 分钟**
以上述示例为指导，加载更多通道，使用 matplotlib 查看它们，并通过切片/步进或使用 matplotlib 的交互式笔记本小部件来探索数据（参见图像左下角的工具栏）。

In [ ]:
# Add your code here

### 坐标切片
我们可以进一步利用Xarray提供的切片功能，根据坐标对数据数组进行切片。我们将在后续教程中学习更高级的版本。在下面的例子中，我们使用以米为单位的X和Y坐标对数据进行切片。

In [ ]:
scn[my_channel].sel(x=slice(-2650000, -2550000), y=slice(4400000, 4200000))

## 访问产品变体
到目前为止，我们一直通过通道的“名称”来引用它们。这些名称是在 Satpy 的读取器中配置的，可能因仪器或甚至同一仪器的不同代际而有所不同。根据您的偏好，您可能希望通过波长来引用产品。并非所有产品都有与之相关的波长，但对于那些有波长的产品，Satpy 为该仪器配置了最小、标称和最大波长的规格。在使用 `.load` 方法加载数据或之后，我们可以使用该范围内的任何波长来访问数据。

In [ ]:
scn[my_channel].attrs['wavelength']  # in µm

In [ ]:
0.47 in scn

In [ ]:
scn[0.485].attrs['name']

数据集还可以通过几种其他常见方式有所不同，例如分辨率（如 1000m 与 500m）、校准（如反射率与辐射率）和极化（如 V 与 H）。在识别产品时，我们也可以使用特殊的 `DatasetID` 对象或在 `.load` 方法中使用关键字参数来指定这些属性。要查看可用数据产品的完整列表，我们可以使用一个新方法 `Scene.available_dataset_ids`。读者可自行练习。

## 地理定位和区域
到目前为止，我们一直将数据作为与地球无关的图像来处理。在Satpy中，我们使用`.attrs['area']`属性中的一个特殊对象来定义数据的位置。
有些人更倾向于在处理数据时使用经度和纬度坐标。在这种情况下，我们可以使用区域来获取经度和纬度数据。

In [ ]:
lons, lats = scn[my_channel].attrs['area'].get_lonlats()
lons

这个函数可能需要一些时间，因为默认情况下它返回一个 NumPy 数组。让我们通过在上面的单元格中指定 `chunks=2048` 关键字参数，让它返回一个 Dask 数组。

In [ ]:
type(scn[my_channel].attrs['area'])

In [ ]:
scn[my_channel].attrs['area']

我们用于ABI数据的“区域”是来自`pyresample`库的`AreaDefinition`对象。这意味着我们的数据在投影表面上是均匀网格化的。这与可能需要通过单独的经度和纬度坐标来处理的非均匀数据不同。
我们将在后面的重采样课程中更详细地了解这两种情况。

## 可用的阅读器
最后，让我们看看Satpy当前还支持哪些其他读取器。我们将使用 `available_readers` 函数来获取可用读取器的列表。

In [ ]:
from satpy import available_readers
sorted(available_readers())

如果缺少某些 Satpy 读取器的依赖项，它们将不会在此处列出。我们可以通过使用 `check_satpy` 来检查缺少哪些功能，就像我们在 [简介](./01_introduction.ipynb) 中所做的那样。